<a href="https://colab.research.google.com/github/mupungijose-hue/Data-Analysis-Projects/blob/main/section08_import_export_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

## Pandas Cheat Sheet: Efficient Data Handling

This notebook demonstrates several powerful Pandas features for streamlined data ingestion and output.

### 1. Streamlined Data Ingestion with `pd.read_csv`

When reading a CSV file, you can perform several data preparation steps directly within `pd.read_csv` to optimize memory usage and simplify your workflow.

**Key Parameters & Chaining:**

*   `header`: Specifies which row to use as the column names (e.g., `0` for the first row, `None` if no header).
*   `names`: Provides a list of custom column names.
*   `skiprows`: Skips rows from the beginning of the file (e.g., `[0]` to skip the first row).
*   `parse_dates`: Automatically parses specified columns as datetime objects.
*   `dtype`: Assigns specific data types to columns for memory efficiency (e.g., `"Int8"`, `"Int16"`, `"Float32"`).
*   `.assign()`: Chains new column creations directly after `read_csv`. Use `lambda x: x['column_name']` to refer to the DataFrame being built.
*   `.astype()`: Further converts data types after column creation, especially useful for new columns.

**Example (from notebook):**

```python
transactions = pd.read_csv(
    "../retail/transactions.csv",
    header=0,
    names=["Date", "Store_Number", "Transaction_Count"],
    skiprows=[0],
    parse_dates=["Date"],
    dtype={
        "Store_Number": "Int8",
        "Transaction_Count": "Int16"
    }
).assign(
    target_pct=lambda x: (x["Transaction_Count"] / 2500),
    met_target=lambda x: (x["Transaction_Count"] / 2500 >= 1),
    bonus_payable=lambda x: (x["Transaction_Count"] / 2500 >= 1) * 100,
    month=lambda x: x["Date"].dt.month,
    day_of_week=lambda x: x["Date"].dt.dayofweek,
).astype({
    "target_pct": "Float32",
    "month": "Int8",
    "day_of_week": "Int8"
})
```

### 2. Checking Memory Usage

Use `.info(memory_usage="deep")` to get a detailed breakdown of your DataFrame's memory consumption before and after transformations, helping you identify and optimize memory hogs.

**Example:**

```python
df.info(memory_usage="deep")
```

### 3. Writing Data to Excel or CSV

**Writing to Multiple Excel Sheets:**

Use `pd.ExcelWriter` as a context manager to write different parts of your DataFrame to separate sheets within a single Excel file.

*   Filter your DataFrame using `.loc` (e.g., `df.loc[df["Date"].dt.year == year]`).
*   Use `.to_excel(writer, sheet_name=str(year))` to write to a specific sheet.

**Example (from notebook):**

```python
with pd.ExcelWriter("DataForChandler.xlsx") as writer:
    for year in range(2013, 2018):
        (transactions
         .loc[transactions["Date"].dt.year == year]
         .to_excel(writer, sheet_name=str(year), index=False))
```

**Writing to Separate CSV Files:**

Loop through your filtered data and use `.to_csv()` to save each segment as a distinct CSV file.

**Example (from notebook):**

```python
for year in range(2013, 2018):
    (transactions
     .loc[transactions["Date"].dt.year == year]
     .to_csv(f"transactions_{year}.csv", index=False))
```

# Assignment 1: Streamlined Data Ingestion

Now that we have a good idea of what we want the data prep on transactions looks like,
let's push that to the read_csv function.

Keep an eye on the memory usage before and after.

* Change the column names to 'Date', 'Store_Number', and 'Transaction_Count'.
* Skip the first row of data.
* Convert columns to the appropriate datatypes.

Then create the columns we created in the assign assignment in Section 3, by chaining assign with read_csv.

Some starter code has been provided for you below. Because the dataframe object returned by read_csv doesn't have a name, we need to use a lambda function to refer to the dataframe.

`transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions["date"].dt.month,
    day_of_week=transactions["date"].dt.dayofweek,
)`

The first one should look like:

`target_pct = lambda x: (x["Transaction_Count"] / 2500)`


In [ ]:
pd.read_csv("../retail/transactions.csv").info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.6 MB


In [ ]:
transactions = pd.read_csv(
    "../retail/transactions.csv",
    header=0,                                                              # Suppress header to allow custom names
    names=["Date", "Store_Number", "Transaction_Count"],                   # Specify new column names
    skiprows=[0],                                                          # Skip the first row of data
    parse_dates=["Date"],                                                  # parse date column
    dtype={"Store_Number": "Int8", "Transaction_Count": "Int16"}).assign(  # Downcast two integer columns
    target_pct = lambda x: (x["Transaction_Count"] / 2500),
    met_target = lambda x: (x["Transaction_Count"] / 2500 >= 1),
    bonus_payable = lambda x: (x["Transaction_Count"] / 2500 >= 1 * 100),
    month = lambda x: x["Date"].dt.month,
    day_of_week = lambda x: x["Date"].dt.dayofweek,
).astype({                                                                 # Cast new columns to correct dtypes.
    "target_pct": "Float32",                                               # Note this could also be done in assign
    "month": "Int8",
    "day_of_week": "Int8"
})


In [ ]:
# Df is significantly reduced in size!

transactions.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83487 entries, 0 to 83486
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               83487 non-null  datetime64[ns]
 1   Store_Number       83487 non-null  Int8          
 2   Transaction_Count  83487 non-null  Int16         
 3   target_pct         83487 non-null  Float32       
 4   met_target         83487 non-null  boolean       
 5   bonus_payable      83487 non-null  boolean       
 6   month              83487 non-null  Int8          
 7   day_of_week        83487 non-null  Int8          
dtypes: Float32(1), Int16(1), Int8(3), boolean(2), datetime64[ns](1)
memory usage: 2.1 MB


# Assignment 2: Write to Excel Sheets

Write the data in the transactions dataframe you created above into an Excel workbook.

Write out a separate sheet for each year of the data.

If you prefer, you can write each year of data to a separate csv file.

In [ ]:
transactions.head()

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week
0,2013-01-02,1,2111,0.8444,False,False,1,2
1,2013-01-02,2,2358,0.9432,False,False,1,2
2,2013-01-02,3,3487,1.3948,True,False,1,2
3,2013-01-02,4,1922,0.7688,False,False,1,2
4,2013-01-02,5,1903,0.7612,False,False,1,2


In [ ]:
transactions.tail()

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week
83482,2017-08-15,50,2804,1.1216,True,False,8,1
83483,2017-08-15,51,1573,0.6292,False,False,8,1
83484,2017-08-15,52,2255,0.902,False,False,8,1
83485,2017-08-15,53,932,0.3728,False,False,8,1
83486,2017-08-15,54,802,0.3208,False,False,8,1


In [ ]:
# Open ExcelWriter to write multiple sheets

with pd.ExcelWriter("DataForChandler.xlsx") as writer:
    for year in range(2013, 2018):                 # Specify years to filter by for each sheet and loop through them
       (transactions
        .loc[transactions["Date"].dt.year == year] # Filter DF to year in current iteration of loop
        .to_excel(writer, sheet_name=str(year)))   # Write each year's DF to sheet named for that year

In [ ]:
for year in range(2013, 2018):                     # Specify years to filter by for each sheet and loop through them
    (transactions
     .loc[transactions["Date"].dt.year == year]    # Filter DF to year in current iteration of loop
     .to_csv(f"transactions_{year}.csv")           # Write each year's DF to sheet named for that year
    )